In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
file_path = '/content/drive/My Drive/metadata.csv'


df = pd.read_csv(file_path)

# Display the first few rows of the DataFrame
df.head()

,examId,seriesNo,aclDiagnosis,kneeLR,roiX,roiY,roiZ,roiHeight,roiWidth,roiDepth,volumeFilename
0,329637,8,0,1,139,184,14,74,72,3,329637-8.pck
1,390116,9,0,0,113,105,10,83,98,6,390116-9.pck
2,404663,8,1,1,120,117,15,101,115,2,404663-8.pck
3,406320,9,0,0,117,124,12,91,80,3,406320-9.pck
4,412857,8,0,1,122,105,14,83,98,4,412857-8.pck


In [4]:
df.head() .style.background_gradient(cmap='Blues')

,examId,seriesNo,aclDiagnosis,kneeLR,roiX,roiY,roiZ,roiHeight,roiWidth,roiDepth,volumeFilename
0,329637,8,0,1,139,184,14,74,72,3,329637-8.pck
1,390116,9,0,0,113,105,10,83,98,6,390116-9.pck
2,404663,8,1,1,120,117,15,101,115,2,404663-8.pck
3,406320,9,0,0,117,124,12,91,80,3,406320-9.pck
4,412857,8,0,1,122,105,14,83,98,4,412857-8.pck


In [5]:
injury_dist = df['aclDiagnosis'].value_counts()
print("Distribution of ACL Diagnosis:\n", injury_dist)

Distribution of ACL Diagnosis:
 aclDiagnosis
0    690
1    172
2     55
Name: count, dtype: int64


In [6]:
df.describe().T .style.background_gradient(cmap='Reds')

,count,mean,std,min,25%,50%,75%,max
examId,917.000000,739320.042530,174745.071850,329637.000000,580637.000000,751087.000000,895401.000000,1028670.000000
seriesNo,917.000000,5.367503,1.082388,3.000000,5.000000,5.000000,5.000000,20.000000
aclDiagnosis,917.000000,0.307525,0.577298,0.000000,0.000000,0.000000,0.000000,2.000000
kneeLR,917.000000,0.511450,0.500142,0.000000,0.000000,1.000000,1.000000,1.000000
roiX,917.000000,114.497274,11.637973,71.000000,107.000000,115.000000,123.000000,146.000000
roiY,917.000000,109.318430,15.617949,22.000000,100.000000,109.000000,118.000000,184.000000
roiZ,917.000000,13.992366,1.408391,9.000000,13.000000,14.000000,15.000000,22.000000
roiHeight,917.000000,91.758997,9.074277,62.000000,86.000000,92.000000,98.000000,124.000000
roiWidth,917.000000,90.736096,10.101242,61.000000,84.000000,91.000000,97.000000,136.000000
roiDepth,917.000000,3.359869,0.733809,2.000000,3.000000,3.000000,4.000000,6.000000


In [7]:
df.sample(5) .style.background_gradient(cmap='Greens')

,examId,seriesNo,aclDiagnosis,kneeLR,roiX,roiY,roiZ,roiHeight,roiWidth,roiDepth,volumeFilename
324,634511,5,0,0,121,104,16,94,97,3,634511-5.pck
898,1022353,5,1,0,109,104,14,109,93,4,1022353-5.pck
877,1009152,5,0,1,102,105,16,104,104,3,1009152-5.pck
237,584196,5,0,0,113,101,13,89,85,3,584196-5.pck
395,703073,5,1,1,115,147,14,79,90,3,703073-5.pck


In [8]:
print(df.isnull().sum())

examId            0
seriesNo          0
aclDiagnosis      0
kneeLR            0
roiX              0
roiY              0
roiZ              0
roiHeight         0
roiWidth          0
roiDepth          0
volumeFilename    0
dtype: int64


In [9]:
#Remove rows containing missing values
df.dropna(inplace=True)

## Check for illogical values
df = df[df['roiHeight'] > 0]
df = df[df['roiWidth'] > 0]
df = df[df['roiDepth'] > 0]

##Ensure no duplicate data
df = df.drop_duplicates()

##Save clean data
df.to_csv("cleaned_dataset.csv", index=False)

print("The data was successfully cleaned!")

The data was successfully cleaned!


In [10]:
##Display and repeat unique values
print(df['aclDiagnosis'].value_counts())
print(df['kneeLR'].value_counts())  # Distribution of affected knee (right/left)

aclDiagnosis
0    690
1    172
2     55
Name: count, dtype: int64
kneeLR
1    469
0    448
Name: count, dtype: int64


In [11]:
correlation = df['roiHeight'].corr(df['roiWidth'])

print(f"Pearson correlation between roiHeight and roiWidth: {correlation}")

Pearson correlation between roiHeight and roiWidth: 0.6658576596548378


In [12]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

X = df[['roiHeight', 'roiWidth', 'roiDepth']]
y = df['aclDiagnosis']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Model coefficients (slopes):", model.coef_)
print("Model intercept:", model.intercept_)

Model coefficients (slopes): [-0.01474846  0.02349512 -0.00174402]
Model intercept: -0.4758248491150287


In [14]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)

r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("R² Score:", r2)

Mean Squared Error: 0.30646360870495115
R² Score: 0.12338358091290769


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report, confusion_matrix

# Define the model
model = Sequential()
model.add(Dense(64, input_dim=3, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

# Train the model with 1000 epochs
history = model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test accuracy:", accuracy)

# Predict values for classification report and confusion matrix
y_pred = (model.predict(X_test) > 0.5)

# Display additional evaluation metrics
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))


Epoch 1/1000
23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.6111 - loss: 3.4911 - val_accuracy: 0.7011 - val_loss: 0.7471
Epoch 2/1000
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5690 - loss: 0.7133 - val_accuracy: 0.7011 - val_loss: 0.7098
Epoch 3/1000
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7325 - loss: 0.5719 - val_accuracy: 0.5815 - val_loss: 0.6152
Epoch 4/1000
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6851 - loss: 0.5911 - val_accuracy: 0.6957 - val_loss: 0.6386
Epoch 5/1000
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7530 - loss: 0.5253 - val_accuracy: 0.6087 - val_loss: 0.6015
Epoch 6/1000
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6815 - loss: 0.5841 - val_accuracy: 0.6848 - val_loss: 0.6072
Epoch 7/1000
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7653 - loss: 0.5450 - val_accuracy: 0.5652 - val_loss: 0.6256
Epoch 8/1000
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6694 - loss: 0.6012 - val_accuracy: 0